In [ ]:
#Import statements
from int_dir.config_file import *
from helper_modules.ai_db_connect import *

import time
import pandas as pd
import numpy as np

In [ ]:
# Temporary script to write any sql table to csv in platform repository
engine = conn_eng()
query_data = "SELECT * FROM intermediate_data.all_bid_table_intrmdt_2020_01_15 where committed_price_ty > 0 and award_price_ty > 0"
raw_data = pd.read_sql(query_data,engine)

raw_data = raw_data[['key','fiscal','destination_no','bid_line_no','customer_no','customer_name',
                     'destination_county_name','destination_state_prov','bid_due_date',
                     'total_qty_ty','price_competitor_id','committed_price_ty','award_price_ty',
                     'award_competitor_id']].drop_duplicates()

In [52]:
len(raw_data.index)

# Base data
base_df = raw_data[['key','fiscal','destination_no','bid_line_no']].drop_duplicates().reset_index(drop=True)
len(base_df.index)

# Basic details for bid
basic_details_df = raw_data[['key','customer_no','customer_name','destination_county_name',
                             'destination_state_prov','bid_due_date','total_qty_ty',
                             'award_price_ty','award_competitor_id']].drop_duplicates().reset_index(drop=True)
len(basic_details_df.index)

# did cmp win bid
cmp_won_df = raw_data.loc[raw_data['price_competitor_id'] == 'CMP',
                          ['key','committed_price_ty','award_competitor_id']].drop_duplicates().reset_index(drop=True)

cmp_won_df['CMP_won_bid'] = np.where(cmp_won_df['award_competitor_id'] == 'CMP', 'Yes', 'No')
cmp_won_df = cmp_won_df.drop(['award_competitor_id'], axis = 1)
cmp_won_df.columns = ['key','cmp_price','cmp_won_bid']

# best competitor
best_comp_df = raw_data.loc[raw_data['price_competitor_id'] != 'CMP',
                          ['key','price_competitor_id','committed_price_ty']].drop_duplicates().reset_index(drop=True)
best_comp_df = best_comp_df.sort_values('committed_price_ty', ascending = False).groupby('key').first().reset_index()
best_comp_df.columns = ['key','top_competitor_id','top_competitor_price']

# lst year
last_yr_df = raw_data[['fiscal','destination_no','bid_line_no','award_price_ty','award_competitor_id']].drop_duplicates().reset_index(drop=True)
last_yr_df['key'] = ((last_yr_df['fiscal']+1).astype(str) + "_" + 
                     last_yr_df['destination_no'].astype(str) + "_" + 
                     last_yr_df['bid_line_no'].astype(str))
last_yr_df = last_yr_df.drop(['fiscal','destination_no','bid_line_no'],axis=1)[['key','award_competitor_id','award_price_ty']]
last_yr_df.columns = ['key','last_yr_award_competitor_id','last_yr_award_price']


# Combining all above tables into one
final_details_df = pd.merge(base_df, basic_details_df,'left',on='key')
final_details_df = pd.merge(final_details_df, cmp_won_df,'left',on='key')
final_details_df = pd.merge(final_details_df, best_comp_df,'left',on='key')
final_details_df = pd.merge(final_details_df, last_yr_df,'left',on='key')

# Adding a few additional columns for shiny output
final_details_df['delta_awardprice_lastyr'] = final_details_df['award_price_ty'] - final_details_df['last_yr_award_price']
final_details_df['incumbent'] = np.where(final_details_df['award_competitor_id'] == final_details_df['last_yr_award_competitor_id'],
                                        final_details_df['award_competitor_id'], 'None')
final_details_df['city_state'] = final_details_df['destination_county_name']+', '+final_details_df['destination_state_prov']

final_details_df.to_csv("/home/ncod/dashboard_input_data/raw_files/key_details_referencetable_0129.csv",index=False)

# there are a few keys with multiple entries in the fina_details_df. Group by and select first() to pick only one item per key


In [ ]:
print(len(base_df.index))
print(len(basic_details_df.index))
print(len(cmp_won_df.index))
print(len(best_comp_df.index))
print(len(last_yr_df.index))
print(len(final_details_df.index))
final_details_df


In [ ]:
start = time.time()
engine = conn_eng()
export_keydetails(engine)
end = time.time()
print(end-start)